### Problem-Independent Machinery

In [ ]:
## The search algorithm itself -- it takes a problem, which will give it an initial state and the goal test,
##   the world state itself which gives it the successor states, and an evaluator that evaluates the quality
##   of a node on the search frontier

#  Priority queue code taken from Pacman project -- PriorityQueue supports
#      pop, isEmpty, and push/update
#
#  Client supplies
#    -- a WorldState; a WorldState implements the method successors()
#    -- a Problem which supplies the initial state and goal state checker
#    -- an Evaluator which supplies a method that evaluates a WorldState
#
#   The search function uses a SearchState which is a WorldState plus a sequence of 
#     actions (not examined by Search).   The search fringe is a priority 
#     queue of SearchState
#
#   Search returns a 2-tuple -- 
#    -- a sequence of actions
#    -- performance information:  process time used, number of nodes expanded, 
#         number of nodes skipped (because they were previously expanded)

from priorityqueue import PriorityQueue
import time

def aStarSearch(problem, evaluator, verbose=None):
    startTime = time.process_time()
    fringe = PriorityQueue()
    visited = {}
    initialWorldState = problem.initial()
    initialValue = evaluator.value(initialWorldState, [])
    initialSearchState = SearchState(initialWorldState, [])
    fringe.update(initialSearchState, initialValue)
    numVisited = numSkipped = 0
    while (True):
        if fringe.isEmpty():
            return (None, (time.process_time() - startTime, numVisited, numSkipped))
        nextNode = fringe.pop()   # A search state (state, actions)
        numVisited += 1
        if (verbose and numVisited % verbose == 0):
            print("Visited " + str(numVisited) + " world is " + str(nextNode._worldState))
            print("Skipped " + str(numSkipped) + " Fringe is size " + str(len(fringe.heap)))
            print("Evaluation is " + str(evaluator.value(nextNode._worldState, nextNode._actions)) + " with actions " + str(len(nextNode._actions)))
        if (problem.isGoal(nextNode.worldState())):
            return (nextNode._actions, (time.process_time() - startTime, numVisited, numSkipped))
        if (nextNode._worldState in visited):
            numSkipped += 1
        else:
            visited[nextNode.worldState()] = True
            successors = nextNode.worldState().successors()
            for successor in successors:
                state, action = successor
                actions = list(nextNode.actions())
                actions.append(action)
                newSS = SearchState(state, actions)
                newValue = evaluator.value(state, actions)
                fringe.update(newSS, newValue)
    raise "Impossible search execution path."

## Instances of SearchState go on the search fringe -- contains both a state and 
## list of actions so far

class SearchState:
    def __str__(self):
        return "{S " + str(self._worldState) + "/" + str(self._actions) + "}"
    
    def __init__(self, worldState, actions):
        self._worldState = worldState
        self._actions = actions
    
    def worldState(self):
        return self._worldState
    
    def actions(self):
        return self._actions

### Interface Between Search and Client ###

In [ ]:
class WorldState:
    #  Method successors() returns a tuple:  (worldState, action)
    def successors():
        raise "Not implemented"

class Problem:
    def __init__(self, initialSpec, goalSpec):
        self.initialSpec = initialSpec
        self.goalSpec = goalSpec
    
    # Method initial returns a world state
    def initial(self):
        raise "Not implemented"
        
    # Method isGoal returns a boolean
    def isGoal(self, state):
        raise "Not implemented"

# Evaluator provides the evaluator f(s) = g(s) + h*(s)

class Evaluator:
    def __init__(self, goalEstimator, actionsCoster):
        self._estimator = goalEstimator
        self._coster = actionsCoster
    def estimateToGoal(self, state):
        return self._estimator(state)
    def costSoFar(self, actions):
        return self._coster(actions)
    def value(self, state, actions):
        return self.estimateToGoal(state) + self.costSoFar(actions)
    

### Domain-Specific Code ###

In [ ]:
import copy

class PancakeWorldState(WorldState):
       
    def __str__(self):
        return "{" + str(self._array) + "}"
    
    def __eq__(self, other):
        if isinstance(other, PancakeWorldState):
            return self._array == other._array 
        else:
            return False

    def __hash__(self):
        return hash(str(self._array) + str(len(self._array))) 
                    
    def successors(self):
        self._length = len(self._array)
        candidates = []
        for i in range(1, self._length):
                candidates.append(self.flip(i))
        candidates = [val for sublist in candidates for val in sublist]
        return candidates
    
    def flip(self, number):
        s = copy.deepcopy(self)
        start = 0
        index = number
        while start < index: 
            temp = s._array[start]
            s._array[start] = s._array[index]
            s._array[index] = temp
            start += 1
            index -= 1
        return [(s, "flip_" + str(number))]
    
    

In [ ]:
class PancakeWorldProblem(Problem):
    def __init__(self, initialSpec):
        for i in initialSpec["array"]:
            if not isinstance(i, int):
                raise TypeError("Only can input digit!")
        super(PancakeWorldProblem, self).__init__(initialSpec, [])
  
    def initial(self):
        state = PancakeWorldState()
        initspec = self.initialSpec
        state._array = initspec["array"]
        return state
    
    def isGoal(self, state):
        result = True
        for i in range(1, len(state._array)):
            if state._array[i] < state._array[i-1]:
                result = False           
        return result

In [ ]:
def uniformCostCoster(actions):
    return len(actions)

In [ ]:
def pancakeSort(array):
    pancakeWorldProblem = PancakeWorldProblem({"array": array})
    bfsEvaluator = Evaluator(lambda state: 0, uniformCostCoster)
    actions, stats = aStarSearch(pancakeWorldProblem, bfsEvaluator, verbose = None)
    print("Expanded " + str(stats[1]))
    return actions

# Extra credit: Heuristic

#### The following is the heuristic function:

In [ ]:
def PancakeWorldEstimator(actualStack):
    cost = 0
    for i in range(0, (len(actualStack) - 1)):
        if (abs(actualStack[i] - actualStack[i+1]) > 1):
            cost += 1
    return cost

#### The following is the new pancakeSort with heuristics:

In [ ]:
def pancakeSort(array):
    pancakeWorldProblem = PancakeWorldProblem({"array": array})
    pancakeWorldEvaluator = Evaluator(lambda state: PancakeWorldEstimator(state._array), uniformCostCoster)
    actions, stats = aStarSearch(pancakeWorldProblem, pancakeWorldEvaluator, verbose = None)
    print("Expanded " + str(stats[1]))
    return actions